In [5]:
# import os
# import re

# repo = r'D:\trufiapp\GANS'
# file_pattern = re.compile(r'^\d{2}-')

# # Obtener la lista de archivos en el directorio
# files = os.listdir(repo)

# # Filtrar solo los archivos de registro que cumplen con el patrón
# log_files = [file for file in files if file_pattern.match(file)]

# # Ordenar los archivos según el prefijo numérico
# log_files.sort(key=lambda x: int(x.split('-')[0]))

# # Mostrar un resumen del contenido de cada archivo
# for log_file in log_files:
#     file_path = os.path.join(repo, log_file)
    
#     # Leemos solo las primeras líneas del archivo para hacer un resumen
#     with open(file_path, 'r') as f:
#         content_summary = ''.join(f.readlines()[:5])  # Puedes ajustar el número de líneas que deseas mostrar

#     print(f"Resumen de contenido de {log_file}:\n{content_summary}\n")


In [6]:
# pip install geopy

In [7]:
import os
import re
import csv
from datetime import datetime
import pytz
from shapely.geometry import shape, Point
import geopandas as gpd
import pandas as pd
from h3 import h3
from geopy.distance import geodesic

In [8]:
# Definir la zona horaria de La Paz, Bolivia
la_paz_timezone = pytz.timezone('America/La_Paz')

repo = r'D:\trufiapp\GANS'
file_pattern = re.compile(r'^\d{2}-')

# Cargar el GeoDataFrame con información de las capitales de Cochabamba
cochabamba_data = gpd.read_file(r'D:\Trufiapp\ETL\Dataset\region_cochabamba_2018\region_cochabamba_2018.geojson')

# Obtener las geometrías de las capitales de Cochabamba
cochabamba_geometries = cochabamba_data['geometry']

def is_point_inside_cochabamba(coords):
    point = Point(coords)
    # Verificar si el punto está dentro de alguna de las capitales de Cochabamba
    return any(point.within(geom) for geom in cochabamba_geometries)

def get_city_from_coords(coords):
    point = Point(coords)
    # Iterar sobre las filas del GeoDataFrame
    for index, row in cochabamba_data.iterrows():
        city_geometry = row['geometry']
        # Verificar si el punto está dentro de la geometría de la ciudad
        if point.within(city_geometry):
            return row['CAPITAL']

    return 'N/A'

def extract_route_info(log_line):
    # Expresión regular para extraer información específica de las solicitudes de rutas
    route_pattern_with_id = re.compile(r'GET /otp/plan\?fromPlace=([-0-9.]+)%2C([-0-9.]+)&toPlace=([-0-9.]+)%2C([-0-9.]+).*?Trufi/.*?/([a-f0-9-]+)')
    route_pattern_without_id = re.compile(r'GET /otp/plan\?fromPlace=([-0-9.]+)%2C([-0-9.]+)&toPlace=([-0-9.]+)%2C([-0-9.]+)')

    match_with_id = route_pattern_with_id.search(log_line)
    match_without_id = route_pattern_without_id.search(log_line)

    # Inicializar variables con valores predeterminados
    origin_latitude = origin_longitude = dest_latitude = dest_longitude = id_user = None

    try:
        if match_with_id:
            origin_latitude, origin_longitude, dest_latitude, dest_longitude, id_user = match_with_id.groups()
        elif match_without_id:
            origin_latitude, origin_longitude, dest_latitude, dest_longitude = match_without_id.groups()
            id_user = 'N/A'  # Asignar un valor predeterminado

        # Verificar si los puntos de origen y destino están dentro de alguna capital de Cochabamba
        if origin_latitude is not None and origin_longitude is not None and dest_latitude is not None and dest_longitude is not None:
            origin_coords = (float(origin_longitude), float(origin_latitude))
            dest_coords = (float(dest_longitude), float(dest_latitude))

            if is_point_inside_cochabamba(origin_coords) and is_point_inside_cochabamba(dest_coords):
                # Obtener la ciudad del origen y destino
                origin_city = get_city_from_coords(origin_coords)
                dest_city = get_city_from_coords(dest_coords)
                # Calcular la distancia entre los puntos de origen y destino en metros
                distance_meters = int(geodesic(origin_coords, dest_coords).meters)

                return origin_latitude, origin_longitude, dest_latitude, dest_longitude, origin_city, dest_city, id_user, distance_meters

    except Exception as e:
        print(f"Error al procesar la línea: {log_line}")
        print(f"Error: {e}")

    return None

def process_log_file(file_path):
    with open(file_path, 'r') as f:
        for line in f:
            if 'GET /otp/plan' in line:
                date_str = re.search(r'\[([^:]+:[^ ]+)', line).group(1)

                # Convertir la fecha a un objeto datetime y añadir la zona horaria de La Paz
                date_time = datetime.strptime(date_str, '%d/%b/%Y:%H:%M:%S').replace(tzinfo=pytz.utc).astimezone(la_paz_timezone)
                
                # Obtener información adicional
                hour = date_time.hour
                day_of_week = date_time.weekday()  # 0: lunes, 1: martes, ..., 6: domingo
                day_of_month = date_time.day
                is_weekend = int(date_time.weekday() in [5, 6])  # 0: no es fin de semana, 1: es fin de semana

                route_info = extract_route_info(line)
                if route_info:
                    origin_latitude, origin_longitude, dest_latitude, dest_longitude, origin_city, dest_city, id_user, distance_meters = route_info
                    
                    yield [date_time.strftime('%Y-%m-%d %H:%M:%S'), hour, day_of_week, day_of_month, is_weekend, origin_latitude, origin_longitude, dest_latitude, dest_longitude, origin_city, dest_city, id_user, distance_meters]

# Obtener la lista de archivos y ordenarlos según el prefijo numérico
files = os.listdir(repo)
log_files = [file for file in files if file_pattern.match(file)]
log_files.sort(key=lambda x: int(x.split('-')[0]))

# Guardar la información en un archivo CSV
csv_file_path = os.path.join(repo, 'route_info.csv')
header = ['date_time', 'hour', 'day_of_week', 'day_of_month', 'is_weekend', 'origin_latitude', 'origin_longitude', 'destination_latitude', 'destination_longitude', 'origin_city', 'dest_city', 'id_user', 'distance_meters']

with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(header)

    for log_file in log_files:
        file_path = os.path.join(repo, log_file)
        route_info_generator = process_log_file(file_path)

        # Escribir las líneas en el archivo CSV
        csv_writer.writerows(route_info_generator)

print(f"La información de las rutas dentro de alguna capital de Cochabamba se ha guardado en {csv_file_path}.")

La información de las rutas dentro de alguna capital de Cochabamba se ha guardado en D:\trufiapp\GANS\route_info.csv.


In [13]:
# import pandas as pd

# repo = r'D:\trufiapp\GANS'
# csv_file_path = os.path.join(repo, 'route_info.csv')

# # Intenta leer el archivo CSV con una codificación diferente
# try:
#     df = pd.read_csv(csv_file_path, parse_dates=['date_time'], encoding='utf-8')
# except UnicodeDecodeError:
#     df = pd.read_csv(csv_file_path, parse_dates=['date_time'], encoding='latin-1')


# # Crear un diccionario que mapea cada índice H3 único a su propio ID único para el origen
# h3_index_to_origin_id = {h3_index: idx + 1 for idx, h3_index in enumerate(df['origin_h3_index'].unique())}

# # Asignar los IDs únicos para el origen utilizando el diccionario
# df['origin_id'] = df['origin_h3_index'].map(h3_index_to_origin_id)

# # Crear un diccionario que mapea cada índice H3 único a su propio ID único para el destino
# h3_index_to_dest_id = {h3_index: idx + 1 for idx, h3_index in enumerate(df['dest_h3_index'].unique())}

# # Asignar los IDs únicos para el destino utilizando el diccionario
# df['destination_id'] = df['dest_h3_index'].map(h3_index_to_dest_id)

# # Crear un nuevo DataFrame solo con las columnas necesarias
# new_df = df[['origin_id', 'origin_h3_index', 'destination_id', 'dest_h3_index']]

# # Guardar el nuevo DataFrame con IDs únicos en columnas adyacentes en el mismo archivo CSV
# df.to_csv(csv_file_path, index=False)

# print(f"La información con IDs únicos se ha guardado en el mismo archivo {csv_file_path} en columnas adyacentes.")
# max_origin_id = df['origin_id'].max()
# max_destination_id = df['destination_id'].max()

# print(f"El máximo ID de origenes es: {max_origin_id}")
# print(f"El máximo ID de destinos es: {max_destination_id}")


In [7]:
# Assuming df is your DataFrame
df_filtered = df[df['distance_meters'] >= 500]
# Imprimir el número de filas y columnas del DataFrame filtrado
print("Número de filas y columnas después de filtrar:", df_filtered.shape)

Número de filas y columnas después de filtrar: (950575, 15)


In [8]:
df_filtered['date_time'] = pd.to_datetime(df_filtered['date_time'])

# Filtra las filas con fechas mayores o iguales a '2022-09-09'
df_filtered = df_filtered[df_filtered['date_time'] >= '2022-09-09']

# Imprime el número de filas y columnas después de aplicar los filtros
print("Número de filas y columnas después de filtrar:", df_filtered.shape)

Número de filas y columnas después de filtrar: (950575, 15)


C:\Users\idaas\AppData\Local\Temp\ipykernel_4128\2439089450.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['date_time'] = pd.to_datetime(df_filtered['date_time'])


In [9]:
# pip install pyspark

In [14]:
# # Supongamos que ID_plus_timestamp y hourly_aggregated son tus DataFrames
# Pick_aggregated = pd.DataFrame()

# # Número de filas por lote
# batch_size = 10000

# # Iterar sobre lotes
# for i in range(0, len(ID_plus_timestamp), batch_size):
#     # Obtener un lote del DataFrame ID_plus_timestamp
#     batch_id_plus_timestamp = ID_plus_timestamp.iloc[i:i+batch_size]
    
#     # Realizar el merge con el lote actual
#     merged_batch = pd.merge(hourly_aggregated, batch_id_plus_timestamp, left_on='UniqueColumn', right_on='UniqueColumn_G', how='right')
    
#     # Agregar el resultado parcial al DataFrame final
#     Pick_aggregated = pd.concat([Pick_aggregated, merged_batch])

# # Reemplazar valores nulos en las columnas PULocationID y Trips_count con 0
# Pick_aggregated['PULocationID'].fillna(0, inplace=True)
# Pick_aggregated['Trips_count'].fillna(0, inplace=True)


In [10]:
# pip install h3

In [24]:
# import os
# import h3
# import folium
# import pandas as pd

# repo = r'D:\trufiapp\GANS'
# csv_file_path = os.path.join(repo, 'route_info - Copy.csv')

# # Cargar datos del archivo CSV en un DataFrame de Pandas
# df = pd.read_csv(csv_file_path)

# # Crear un mapa con folium
# m = folium.Map(location=[df['origin_latitude'].mean(), df['origin_longitude'].mean()], zoom_start=10)

# # Agregar hexágonos al mapa para origenes
# for _, row in df.iterrows():
#     lat, lon = row['origin_latitude'], row['origin_longitude']
#     hexagon = h3.h3_to_geo_boundary(row['origin_h3_index'])
#     folium.Polygon(locations=hexagon, color='blue', fill=True, fill_color='blue', fill_opacity=0.4).add_to(m)

# # Guardar el mapa de origenes como un archivo HTML
# map_origin_file_path = os.path.join(repo, 'h3_map_origin.html')
# m.save(map_origin_file_path)

# # Crear un nuevo mapa con folium para destinos
# m_dest = folium.Map(location=[df['destination_latitude'].mean(), df['destination_longitude'].mean()], zoom_start=10)

# # Agregar hexágonos al mapa para destinos
# for _, row in df.iterrows():
#     lat, lon = row['destination_latitude'], row['destination_longitude']
#     hexagon = h3.h3_to_geo_boundary(row['dest_h3_index'])
#     folium.Polygon(locations=hexagon, color='red', fill=True, fill_color='red', fill_opacity=0.4).add_to(m_dest)

# # Guardar el mapa de destinos como un archivo HTML
# map_dest_file_path = os.path.join(repo, 'h3_map_dest.html')
# m_dest.save(map_dest_file_path)

# print(f"Los mapas con hexágonos H3 para origenes y destinos se han guardado en {map_origin_file_path} y {map_dest_file_path} respectivamente.")


El mapa con hexágonos H3 se ha guardado en D:\trufiapp\GANS\h3_map.html.


In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.linear_model import LinearRegression
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# import matplotlib.pyplot as plt

# # Leer tus datos
# # Cambia 'ruta_de_tu_archivo.csv' al nombre de tu archivo
# df = pd.read_csv('ruta_de_tu_archivo.csv')

# # Seleccionar características y objetivo
# features = ['hour', 'day_of_week', 'day_of_month', 'is_weekend', 'origin_id', 'destination_id']
# target = 'distance_meters'

# X = df[features]
# y = df[target]

# # Dividir los datos en conjuntos de entrenamiento y prueba
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# # Escalar los datos
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Entrenar un modelo de regresión lineal
# linear_reg_model = LinearRegression()
# linear_reg_model.fit(X_train_scaled, y_train)

# # Realizar predicciones en el conjunto de prueba
# y_pred = linear_reg_model.predict(X_test_scaled)

# # Evaluar el rendimiento del modelo
# model_metrics = get_metrics('Linear Regression', y_test, y_pred)

# # Comparar métricas con otros modelos (puedes agregar más modelos según sea necesario)
# compare_model_metrics([model_metrics])

# # Visualizar algunas predicciones
# plot_real_vs_pred('Linear Regression', y_test[:40], y_pred[:40])
